In [1]:
#  Imports

import numpy as np
import pandas as pd

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RZZGate
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

from scipy.optimize import minimize

np.random.seed(42)


In [2]:
# Cell 3: Portfolio data (6 assets)

n_assets = 6

# Retornos esperados (ejemplo sintético)
mu = np.array([0.10, 0.12, 0.08, 0.15, 0.07, 0.11])

# Matriz de covarianzas sintética (simétrica, "realista")
Sigma = np.array([
    [0.020, 0.018, 0.012, 0.015, 0.010, 0.011],
    [0.018, 0.030, 0.016, 0.020, 0.014, 0.015],
    [0.012, 0.016, 0.025, 0.018, 0.013, 0.014],
    [0.015, 0.020, 0.018, 0.035, 0.017, 0.019],
    [0.010, 0.014, 0.013, 0.017, 0.022, 0.016],
    [0.011, 0.015, 0.014, 0.019, 0.016, 0.028],
])

lambda_risk = 5.0     # peso del riesgo
A_penalty   = 2.0     # peso de penalización de cardinalidad
k_max       = 3       # máximo número de activos en el portafolio


In [3]:
# Build QUBO matrix Q and cost function

def build_qubo(mu, Sigma, lambda_risk, A, k_max):
    n = len(mu)
    Q = np.zeros((n, n))

    # Parte de retorno: -mu_i x_i
    for i in range(n):
        Q[i, i] += -mu[i]

    # Parte de riesgo: lambda * sum_{i,j} Sigma_ij x_i x_j
    for i in range(n):
        for j in range(i, n):
            Q[i, j] += lambda_risk * Sigma[i, j]

    # Penalización de cardinalidad: A (sum x_i - k_max)^2
    # = A ( sum x_i + 2 sum_{i<j} x_i x_j - 2 k_max sum x_i + k_max^2 )
    for i in range(n):
        Q[i, i] += A * (1 - 2 * k_max)  # término lineal
    for i in range(n):
        for j in range(i+1, n):
            Q[i, j] += 2 * A            # término cuadrático x_i x_j

    return Q

Q = build_qubo(mu, Sigma, lambda_risk, A_penalty, k_max)

def qubo_cost(x, Q):
    x = np.array(x)
    return float(x @ Q @ x)


In [4]:
# QUBO to Ising parameters

def qubo_to_ising(Q):
    n = Q.shape[0]
    h = np.zeros(n)
    J = np.zeros((n, n))
    constant = 0.0

    # Diagonal
    for i in range(n):
        qii = Q[i, i]
        if abs(qii) < 1e-12:
            continue
        # x_i = (1 - Z_i)/2
        constant += qii * 0.5
        h[i]     += -qii * 0.5

    # Off-diagonal
    for i in range(n):
        for j in range(i+1, n):
            q = Q[i, j]
            if abs(q) < 1e-12:
                continue
            # x_i x_j = (1/4)(1 - Z_i - Z_j + Z_i Z_j)
            constant += q * 0.25
            h[i]     += -q * 0.25
            h[j]     += -q * 0.25
            J[i, j]  +=  q * 0.25

    return constant, h, J

const_shift, h, J = qubo_to_ising(Q)

print("Constante:", const_shift)
print("h:", h)
print("J (triángulo superior):")
print(J)


Constante: -14.629999999999995
h: [-0.0825  -0.11875 -0.11375 -0.12375 -0.1075  -0.10875]
J (triángulo superior):
[[0.      1.0225  1.015   1.01875 1.0125  1.01375]
 [0.      0.      1.02    1.025   1.0175  1.01875]
 [0.      0.      0.      1.0225  1.01625 1.0175 ]
 [0.      0.      0.      0.      1.02125 1.02375]
 [0.      0.      0.      0.      0.      1.02   ]
 [0.      0.      0.      0.      0.      0.     ]]


In [5]:
# Classical brute-force optimum

def brute_force_optimum(Q):
    n = Q.shape[0]
    best_x = None
    best_cost = np.inf

    for mask in range(2**n):
        bits = [(mask >> i) & 1 for i in range(n)]
        c = qubo_cost(bits, Q)
        if c < best_cost:
            best_cost = c
            best_x = bits

    return np.array(best_x), best_cost

x_opt_classical, cost_opt_classical = brute_force_optimum(Q)

def portfolio_stats(x, mu, Sigma):
    x = np.array(x)
    ret = float(mu @ x)
    var = float(x @ Sigma @ x)
    return ret, var

ret_classical, var_classical = portfolio_stats(x_opt_classical, mu, Sigma)

print("Mejor solución clásica:")
print("x* =", x_opt_classical)
print("Cost* =", cost_opt_classical)
print("Retorno* =", ret_classical)
print("Varianza* =", var_classical)


Mejor solución clásica:
x* = [1 0 0 0 1 1]
Cost* = -17.745
Retorno* = 0.28
Varianza* = 0.144


In [6]:
# AerSimulator backend (Qiskit 1.0 style)

simulator = AerSimulator()


In [7]:
#  QAOA circuit constructor (p layers)

def qaoa_circuit(params, p, h, J):
    """
    params: array of length 2p -> [gamma_1...gamma_p, beta_1...beta_p]
    """
    n = len(h)
    gammas = params[:p]
    betas  = params[p:]

    qc = QuantumCircuit(n)
    # Estado inicial |+>^{⊗n}
    qc.h(range(n))

    for layer in range(p):
        gamma = gammas[layer]
        beta  = betas[layer]

        # Cost Hamiltonian: h_i Z_i
        for i in range(n):
            if abs(h[i]) > 1e-12:
                qc.rz(2 * gamma * h[i], i)

        # Cost Hamiltonian: J_ij Z_i Z_j
        for i in range(n):
            for j in range(i+1, n):
                if abs(J[i, j]) > 1e-12:
                    qc.append(RZZGate(2 * gamma * J[i, j]), [i, j])

        # Mixer: sum X_i -> RX(2 beta)
        for i in range(n):
            qc.rx(2 * beta, i)

    qc.measure_all()
    return qc


In [8]:
# Expectation value with AerSimulator

def counts_to_expectation(counts, Q):
    shots = sum(counts.values())
    exp_val = 0.0
    for bitstring, c in counts.items():
        # Qiskit devuelve '010101' con qubit 0 a la derecha
        bits = [int(b) for b in bitstring[::-1]]
        exp_val += qubo_cost(bits, Q) * c
    return exp_val / shots

def qaoa_expectation(params, p, h, J, Q, shots=4096):
    qc = qaoa_circuit(params, p, h, J)
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    return counts_to_expectation(counts, Q)


In [9]:
# Parameter optimization for QAOA

def run_qaoa(p, h, J, Q, initial_params=None):
    if initial_params is None:
        initial_params = 0.1 * np.random.randn(2 * p)

    def objective(params):
        return qaoa_expectation(params, p, h, J, Q)

    result = minimize(
        objective,
        x0=initial_params,
        method="COBYLA",
        options={"maxiter": 80, "disp": True}
    )
    return result

print("=== QAOA p = 1 ===")
res_p1 = run_qaoa(1, h, J, Q)

print("\n=== QAOA p = 2 ===")
res_p2 = run_qaoa(2, h, J, Q)


=== QAOA p = 1 ===
Return from COBYLA because the trust region radius reaches its lower bound.
Number of function values = 36   Least value of F = -16.64378295898438
The corresponding X is: [ 1.35946096 -0.23221132]


=== QAOA p = 2 ===
Return from COBYLA because the trust region radius reaches its lower bound.
Number of function values = 42   Least value of F = -16.724796142578118
The corresponding X is:
[-0.08089434 -0.09607669  1.05737797 -1.07775424]



In [ ]:
# Extract best bitstring from optimized parameters

def best_bitstring_from_params(params, p, h, J, shots=4096):
    qc = qaoa_circuit(params, p, h, J)
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    best_bs = max(counts, key=counts.get)
    bits = [int(b) for b in best_bs[::-1]]
    return bits, counts

# p = 1
x_qaoa_p1, counts_p1 = best_bitstring_from_params(res_p1.x, 1, h, J)
cost_p1 = qubo_cost(x_qaoa_p1, Q)
ret_p1, var_p1 = portfolio_stats(x_qaoa_p1, mu, Sigma)

# p = 2
x_qaoa_p2, counts_p2 = best_bitstring_from_params(res_p2.x, 2, h, J)
cost_p2 = qubo_cost(x_qaoa_p2, Q)
ret_p2, var_p2 = portfolio_stats(x_qaoa_p2, mu, Sigma)

print("=== QAOA p = 1 ===")
print("Bitstring:", x_qaoa_p1)
print("Cost:", cost_p1)
print("Return:", ret_p1)
print("Variance:", var_p1)

print("\n=== QAOA p = 2 ===")
print("Bitstring:", x_qaoa_p2)
print("Cost:", cost_p2)
print("Return:", ret_p2)
print("Variance:", var_p2)

print("\n=== Classical optimum ===")
print("Bitstring:", x_opt_classical)
print("Cost:", cost_opt_classical)
print("Return:", ret_classical)
print("Variance:", var_classical)
